In [17]:
!pip install streamlit
!pip install google-generativeai
!pip install python-dotenv
!pip install langchain
!pip install PyPDF2
!pip install chromadb
!pip install faiss-cpu
!pip install langchain_google_genai
!pip install langchain_text_splitters
!pip install langchain_community

In [32]:
import streamlit as st
import google.generativeai as genai
import os
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
LANGCHAIN_API_KEY="lsv2_pt_80055a8dc0494a73b9e256d042a108f8_8e9fe17"
GOOGLE_API_KEY="AIzaSyA1i0PoAjJ5txJaebB1MfeRgHw55SiC"
LANGCHAIN_PROJECT="CHATBOT"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
genai.configure(api_key="AIzaSyA1i0PoAjJ5txJaebB1MfeRgHw55SiC")

In [33]:
#fn to extract text
def get_pdf_text(pdf_docs):
  text=""
  for pdf in pdf_docs:
    pdf_reader=PdfReader(pdf)
    for page in pdf_reader.pages:
      text+=page.extract_text()
  return text


In [34]:
#fn to split text
def get_text_chunks(text):
  text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,

  )
  chunks=text_splitter.split_text(text)
  return chunks

In [35]:
#fn to embed using faiss
!pip install langchain_google_genai
def get_vector_store(text_chunks):
  embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001')
  vector_store=FAISS.from_texts(texts=text_chunks,embedding=embeddings)
  vector_store.save_local("faiss_index")
  print(embeddings)
#

In [36]:
def conversation_chain():
  prompt_template = """Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

  model=ChatGoogleGenerativeAI(model="gemini-1.0-pro",temperature=0.3)
  prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])
  chain=load_qa_chain(llm=model,chain_type="stuff",prompt=prompt)
  return chain
#

In [42]:
def user_input(user_question):
  embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
  new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
  docs = new_db.similarity_search(user_question)
  chain = conversation_chain()
  response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
  return response["output_text"]


In [48]:

if __name__ == "__main__":
    # Example of processing PDFs (replace 'pdf_docs' with actual file paths)
    pdf_docs = ["/content/Harrier_OSB.pdf", "/content/grand-i10-nios.pdf","/content/taigun-owners-manual.pdf"]
    raw_text = get_pdf_text(pdf_docs)
    text_chunks = get_text_chunks(raw_text)
    get_vector_store(text_chunks)

    # Example of querying the vector store
    question = "What is Automatic activation of Immobilizer?"
    answer = user_input(question)
    print(f"Answer: {answer}")

client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7cb54b784bb0> model='models/embedding-001' task_type=None google_api_key=SecretStr('**********') credentials=None client_options=None transport=None request_options=None
Answer: If key is removed from ignition switch , the engine will be immobilized automatically even if you forget to lock the vehicle.
